# **Coursera Capstone Project**

# ***The Battle of Neighborhoods (Week 1)***

#**1.Introduction To Problem**

### Business Problem:

The challenge is to find a suitable apartment for rent near Hyderabad that complies with the demands on location, price and venues. The data required to resolve this challenge is described in the following section 2, below.

# **2.Data Section**

### Description of the data and its sources that will be used to solve the problem

### Description of the Data

The following data is required to answer the issues of the problem:

* List of Boroughs and neighborhoods of secunderabad with their geodata (latitude and longitude)
* List of Subway metro stations in secunderabad with their address location
* List of apartments for rent in secunderabad area with their addresses and price
* Preferably, a list of apartment for rent with additional information, such as price, address, area, # of beds, etc
* Venues for each secunderabad neighborhood ( than can be clustered)
* Venues for subway metro stations, as needed

# **Solution for the problem**

In [1]:
import numpy as np # library to handle data in a vectorized manner
import time
import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files
import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

/bin/bash: conda: command not found
Libraries imported.


In [24]:
address = 'secunderabad,IN'

geolocator = Nominatim()
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Secundrabad(Hyderabad),India are {}, {}.'.format(latitude, longitude))

/usr/local/lib/python3.6/dist-packages/geopy/geocoders/osm.py:143: UserWarning: Using Nominatim with the default "geopy/1.17.0" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  UserWarning


The geograpical coordinate of Secundrabad(Hyderabad),India are 17.4435509, 78.4961766.


In [17]:

neighborhood_latitude=17.434227
neighborhood_longitude=78.386659

In [18]:
LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 500 # define radius
CLIENT_ID = 'VWREFNGMYCW5LX54RV42VS4G1U1Z4QJ55JY3NXAZXCRGYNUS' # your Foursquare ID
CLIENT_SECRET = '555Q0NH0RPQ5LVZPZ2Q1JBOYGRBBNKY4O222GRATAPCQQFQS' # your Foursquare Secret
VERSION = '20180604'
# create URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url # display URL

'https://api.foursquare.com/v2/venues/explore?&client_id=VWREFNGMYCW5LX54RV42VS4G1U1Z4QJ55JY3NXAZXCRGYNUS&client_secret=555Q0NH0RPQ5LVZPZ2Q1JBOYGRBBNKY4O222GRATAPCQQFQS&v=20180604&ll=17.434227,78.386659&radius=500&limit=100'

In [19]:
results = requests.get(url).json()
#results

In [20]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [21]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head(10)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  This is separate from the ipykernel package so we can avoid doing imports until


,name,categories,lat,lng
0,Cream Stone Creations,Ice Cream Shop,17.434191,78.386829
1,Inorbit Mall,Shopping Mall,17.433610,78.386207
2,Chutneys,Vegetarian / Vegan Restaurant,17.434254,78.386864
3,KFC,Fast Food Restaurant,17.434201,78.386809
4,Fusion 9,Lounge,17.434292,78.386714
5,News Cafe,Cocktail Bar,17.434421,78.386998
6,Food Court @ Inorbit Mall,Food Court,17.434199,78.386831
7,4 seasons,Indian Restaurant,17.434258,78.386713
8,Shoppers Stop,Shopping Mall,17.434255,78.386849
9,BeerHouse,Beer Garden,17.434291,78.386993


In [29]:
map_hyd = folium.Map(location=[latitude, longitude], zoom_start=17)

# add markers to map
for lat, lng, label in zip(nearby_venues['lat'], nearby_venues['lng'], nearby_venues['name']):
    label = folium.Popup(label, parse_html=True)
    folium.RegularPolygonMarker(
        [lat, lng],
        number_of_sides=4,
        radius=10,
        popup=label,
        color='blue',
        fill_color='#0f0f0f',
        fill_opacity=0.7,
    ).add_to(map_sg)  
    
map_hyd